In [1]:
# Import necessary libraries
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu

c:\Users\humphrys\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load preprocessed data
cleaned_data = pd.read_csv(r"C:/Project1/ForkNatNew/data/processed/cleaned_data.csv")

In [3]:
# Load trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('models/trained_gpt2_model')
tokenizer = GPT2Tokenizer.from_pretrained('models/trained_gpt2_tokenizer')

In [4]:
# Set pad token to eos token and padding side to left
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

In [5]:
# Prepare evaluation data
texts = [" ".join(tokens) for tokens in cleaned_data['clean_text']]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

In [6]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
# Generate predictions
model.eval()
with torch.no_grad():
    outputs = model.generate(inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device), max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [8]:
# Decode predictions
predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [9]:
# Example reference text for BLEU score evaluation
reference_texts = ["I hope this email finds you well."] * len(predictions)  # Ensure the list is long enough

# Calculate BLEU scores
for i, prediction in enumerate(predictions):
    reference = [reference_texts[i].split()]  # Tokenize reference text
    candidate = prediction.split()  # Tokenize generated text
    bleu_score = sentence_bleu(reference, candidate)
    print(f"Prediction {i+1}: {prediction}")
    print(f"BLEU score: {bleu_score}")

Prediction 1: [ ' e m a i l ' ,   ' s u b j e c t ' ,   ' a n n u a l ' ,   ' i m p a c t ' ,   ' f a c t o r ' ,   ' j o u r n a l ' ,   ' e m a i l ' ,   ' b o d y ' ,   ' d e a r ' ,   ' p r o m i s e d ' ,   ' a t t a c h i n g ' ,   ' p a c k e t ' ,   ' a d d i t i o n a l ' ,   ' i n f o r m a t i o n ' ,   ' r e l a t e d ' ,   ' i m p a c t ' ,   ' f a c t o r ' ,   ' j o u r n a l ' ,   ' r a n k i n g ' ,   ' j o u r n a l ' ,   ' f e e l ' ,   ' f r e e ' ,   ' t o u c h ' ,   ' q u e s t i o n s ' ,   ' i n f o r m a t i o n ' ,   ' c o n t a i n e d ' ,   ' s i n c e r e l y ' ,   ' p u b ' ,   ' e d ' ]
BLEU score: 0
Prediction 2: [ ' e m a i l ' ,   ' s u b j e c t ' ,   ' s o c i e t y ' ,   ' c o n t r a c t ' ,   ' r e n e w a l ' ,   ' e m a i l ' ,   ' b o d y ' ,   ' h i ' ,   ' n a m e ' ,   ' h o p e ' ,   ' w e l l ' ,   ' c o n t r a c t ' ,   ' s a g e ' ,   ' s o c i e t y ' ,   ' d u e ' ,   ' e x p i r e ' ,   ' d a t e ' ,   ' p e r ' ,   ' c o n t r a c 